In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_input_data

import cnn_utils

In [2]:
# Create an interactive session.
sess = tf.InteractiveSession()

In [3]:
# Load the dataset.
dataset = mnist_input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
"""Define the network."""

# Input and output placeholders.
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Build the network.
keep_prob = tf.placeholder(tf.float32)
y = cnn_utils.inference(x, keep_prob)
loss = cnn_utils.loss(y, y_)
accuracy = cnn_utils.accuracy(y, y_)

# Setup training.
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = cnn_utils.training(loss, accuracy, optimizer)
saver = tf.train.Saver()

summary = tf.merge_all_summaries()
summary_writer = tf.train.SummaryWriter("checkpoint", sess.graph)

In [5]:
# Perform the training.
sess.run(tf.initialize_all_variables())
validation_feed_dict = {
    x:dataset.validation.images,
    y_: dataset.validation.labels,
    keep_prob: 1.0
}
for i in range(20000):
    batch = dataset.train.next_batch(50)
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if (i+1) % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        validation_accuracy = accuracy.eval(feed_dict=validation_feed_dict)
        print("step %d, training accuracy %.2f%%, validation accuracy %.2f%%" %
              (i+1, train_accuracy * 100, validation_accuracy * 100))
        saver.save(sess, "checkpoint/cnn", global_step = i+1)
        
        summary_str = sess.run(summary, feed_dict=validation_feed_dict)
        summary_writer.add_summary(summary_str, i+1)
        summary_writer.flush()

step 100, training accuracy 86.00%, validation accuracy 81.98%
step 200, training accuracy 92.00%, validation accuracy 89.30%
step 300, training accuracy 92.00%, validation accuracy 91.40%
step 400, training accuracy 90.00%, validation accuracy 93.72%
step 500, training accuracy 88.00%, validation accuracy 94.08%
step 600, training accuracy 98.00%, validation accuracy 94.80%
step 700, training accuracy 98.00%, validation accuracy 94.92%
step 800, training accuracy 90.00%, validation accuracy 95.32%
step 900, training accuracy 90.00%, validation accuracy 96.10%
step 1000, training accuracy 96.00%, validation accuracy 96.32%
step 1100, training accuracy 100.00%, validation accuracy 96.04%
step 1200, training accuracy 94.00%, validation accuracy 96.58%
step 1300, training accuracy 98.00%, validation accuracy 96.70%
step 1400, training accuracy 96.00%, validation accuracy 97.08%
step 1500, training accuracy 100.00%, validation accuracy 97.22%
step 1600, training accuracy 96.00%, validation

In [6]:
# Run inference and get accuracy.
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: dataset.test.images, y_: dataset.test.labels, keep_prob: 1.0}))

test accuracy 0.9912
